<h1 div class='alert alert-success'><center> Features Engineering </center></h1>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/26480/logos/header.png?t=2021-04-09-00-57-05)

# <div class="alert alert-success">  OBJETIVO </div> 

O objetivo neste notebook é criação novas variáveis (feature) que possam ajudar na identificação de novos padrões, para ajudar os classificadore (modelos) nas previsões. 

# <div class="alert alert-success">  1. IMPORTAÇÕES </div> 

## 1.1. Instalações

In [ ]:
! pip install --q scikit-plot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1.2. Bibliotecas 

In [1]:
import warnings
import random
import os
import gc
import torch
import math

In [2]:
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt 
import seaborn           as sns
import joblib            as jb
import xgboost           as xgb
import scikitplot        as skplt

In [3]:
from sklearn.model_selection import train_test_split,  KFold, StratifiedKFold
from sklearn.preprocessing   import StandardScaler, MinMaxScaler, RobustScaler 
from sklearn.preprocessing   import MaxAbsScaler, QuantileTransformer, LabelEncoder, normalize
from sklearn.impute          import SimpleImputer
from sklearn                 import metrics
from datetime                import datetime
from sklearn.cluster         import KMeans
from sklearn.decomposition   import PCA

In [4]:
from yellowbrick.cluster        import KElbowVisualizer, SilhouetteVisualizer
from sklearn.utils.class_weight import compute_sample_weight
from scipy                      import stats
from scipy.cluster              import hierarchy as hc

## 1.3. Funções
Aqui centralizamos todas as funções desenvolvidas durante o projeto para melhor organização do código.

In [5]:
def jupyter_setting():
    
    %matplotlib inline
      
    #os.environ["WANDB_SILENT"] = "true" 
    #plt.style.use('bmh') 
    #plt.rcParams['figure.figsize'] = [20,15]
    #plt.rcParams['font.size']      = 13
     
    pd.options.display.max_columns = None
    #pd.set_option('display.expand_frame_repr', False)

    warnings.filterwarnings(action='ignore')
    warnings.simplefilter('ignore')
    warnings.filterwarnings('ignore')
    #warnings.filterwarnings(category=UserWarning)

    warnings.filterwarnings('ignore', category=DeprecationWarning)
    warnings.filterwarnings('ignore', category=FutureWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    #warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

    pd.set_option('display.max_rows', 150)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.max_colwidth', None)

    icecream = ["#00008b", "#960018","#008b00", "#00468b", "#8b4500", "#582c00"]
    #sns.palplot(sns.color_palette(icecream))
    
    return icecream

icecream = jupyter_setting()

# Colors
dark_red = "#b20710"
black    = "#221f1f"
green    = "#009473"
myred    = '#CD5C5C'
myblue   = '#6495ED'
mygreen  = '#90EE90'

cols= [myred, myblue,mygreen]

In [6]:
colors = ["lightcoral", "sandybrown", "darkorange", "mediumseagreen",
          "lightseagreen", "cornflowerblue", "mediumpurple", "palevioletred",
          "lightskyblue", "sandybrown", "yellowgreen", "indianred",
          "lightsteelblue", "mediumorchid", "deepskyblue"]

In [7]:
def reduce_memory_usage(df, verbose=True):
    
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    
    for col in df.columns:
        
        col_type = df[col].dtypes
        
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
        
    return df

In [8]:
def missing_zero_values_table(df):
        mis_val         = df.isnull().sum()
        mis_val_percent = round(df.isnull().mean().mul(100), 2)
        mz_table        = pd.concat([mis_val, mis_val_percent], axis=1)
        mz_table        = mz_table.rename(columns = {df.index.name:'col_name', 
                                                     0 : 'Valores ausentes', 
                                                     1 : '% de valores totais'})
        
        mz_table['Tipo de dados'] = df.dtypes
        mz_table                  = mz_table[mz_table.iloc[:,1] != 0 ]. \
                                     sort_values('% de valores totais', ascending=False)
        
        msg = "Seu dataframe selecionado tem {} colunas e {} " + \
              "linhas. \nExistem {} colunas com valores ausentes."
            
        print (msg.format(df.shape[1], df.shape[0], mz_table.shape[0]))
        
        return mz_table.reset_index()

In [9]:
def scaler_MaxAbsScaler_StandardScaler(df):    
    sc_mm = MaxAbsScaler()
    sc_st = StandardScaler()     
    col = df.columns
    df  = sc_mm.fit_transform(df)
    df  = pd.DataFrame(sc_st.fit_transform(df), columns=col)    
    return df

In [ ]:
def describe(df):
    var = df.columns

    # Medidas de tendência central, média e mediana 
    ct1 = pd.DataFrame(df[var].apply(np.mean)).T
    ct2 = pd.DataFrame(df[var].apply(np.median)).T

    # Dispensão - str, min , max range skew, kurtosis
    d1 = pd.DataFrame(df[var].apply(np.std)).T
    d2 = pd.DataFrame(df[var].apply(min)).T
    d3 = pd.DataFrame(df[var].apply(max)).T
    d4 = pd.DataFrame(df[var].apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(df[var].apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(df[var].apply(lambda x: x.kurtosis())).T
    d7 = pd.DataFrame(df[var].apply(lambda x: (3 *( np.mean(x) - np.median(x)) / np.std(x) ))).T

    # concatenete 
    m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6, d7]).T.reset_index()
    m.columns = ['attrobutes', 'min', 'max', 'range', 'mean', 'median', 'std','skew', 'kurtosis','coef_as']
    
    return m

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, "r-", label=label)
    ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c=".3")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.rcParams['font.size'] = 12
    plt.title('ROC curve for TPS 09')
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.legend(loc="lower right")
    plt.grid(True)

In [ ]:
def confusion_plot(matrix, labels = None, title = None):
        
    labels = labels if labels else ['Negative (0)', 'Positive (1)']    
    
    fig, ax = plt.subplots(nrows=1, ncols=1)
    
    sns.heatmap(data        = matrix, 
                cmap        = 'Blues', 
                annot       = True, 
                fmt         = 'd',
                xticklabels = labels, 
                yticklabels = labels, 
                ax          = ax);
    
    ax.set_xlabel('\n PREVISTO', fontsize=15)
    ax.set_ylabel('REAL \n', fontsize=15)
    ax.set_title(title)
    
    plt.close();
    
    return fig;

In [ ]:
def df_corr(df, annot_=False):
    
    df = df.corr(method ='pearson').round(5)

    # Máscara para ocultar a parte superior direita do gráfico, pois é uma duplicata
    mask = np.zeros_like(df)
    mask[np.triu_indices_from(mask)] = True

    # Making a plot
    plt.figure(figsize=(15,12))
    ax = sns.heatmap(df, annot=annot_, mask=mask, cmap="RdBu", annot_kws={"weight": "bold", "fontsize":13})

    ax.set_title("Mapa de calor de correlação das variável", fontsize=17)

    plt.setp(ax.get_xticklabels(), 
             rotation      = 90, 
             ha            = "right",
             rotation_mode = "anchor", 
             weight        = "normal")

    plt.setp(ax.get_yticklabels(), 
             weight        = "normal",
             rotation_mode = "anchor", 
             rotation      = 0, 
             ha            = "right");

In [ ]:
def graf_outlier(df, feature):
    col = [(0,4), (5,9)]

    df_plot = ((df[feature] - df[feature].min())/
               (df[feature].max() - df[feature].min()))

    fig, ax = plt.subplots(len(col), 1, figsize=(15,7))

    for i, (x) in enumerate(col): 
        sns.boxplot(data = df_plot.iloc[:, x[0]:x[1] ], ax = ax[i]); 

In [ ]:
def diff(t_a, t_b):
    from dateutil.relativedelta import relativedelta
    t_diff = relativedelta(t_b, t_a)  # later/end time comes first!
    return '{h}h {m}m {s}s'.format(h=t_diff.hours, m=t_diff.minutes, s=t_diff.seconds)

In [ ]:
def free_gpu_cache():
    
    # https://www.kaggle.com/getting-started/140636
    #print("Initial GPU Usage")
    #gpu_usage()                             

    #cuda.select_device(0)
    #cuda.close()
    #cuda.select_device(0)   
    
    gc.collect()
    torch.cuda.empty_cache()

## 1.4. Dataset

## 1.4.1. Descrição das variáveis

A área de estudo inclui quatro áreas selvagens localizadas na Floresta Nacional Roosevelt, no norte do Colorado. Cada observação é um patch de 30m x 30m. 

O conjunto de treinamento (train.csv) tem 4.000.000 observações e 53 variáveis, a variáveis que vamos prever é `Cover_Type` para o conjunto de teste (test.csv) que tem 1.000.000 observações.

Os sete tipos que contém a variável `Cover_Type` são:

1. Spruce/Fir
2. Lodgepole Pine
3. Ponderosa Pine
4. Cottonwood/Willow
5. Aspen
6. Douglas-fir
7. Krummholz

Variáveis do dataset:

- **Elevation**: Elevação em metros;
- **Aspect**: Aspecto em graus azimuth;
- **Slope**: Inclinação em graus;
- **Horizontal_Distance_To_Hydrology**: Horz Dist para variável de água de superfície mais próximos;
- **Vertical_Distance_To_Hydrology**: Vert Dist para recursos de água de superfície mais próximos;
- **Horizontal_Distance_To_Roadways**: Horz Dist para a estrada mais próxima;
- **Hillshade_9am**: índice Hillshade às 9h com valores entre 0 e 255;
- **Hillshade_Noon**: índice Hillshade ao meio-dia com valores entre 0 e 255;
- **Hillshade_3pm**: índice Hillshade às 3pm com valores entre 0 e 255;
- **Horizontal_Distance_To_Fire_Points**: Horz Dist para os pontos de ignição do incêndio florestal mais próximos;
- **Wilderness_Area**: (4 colunas binárias, 0 = ausência ou 1 = presença) - Designação de área selvagem;
- **Soil_Type**: (40 colunas binárias, 0 = ausência ou 1 = presença) - Designação do tipo de solo;
- **Cover_Type**: (7 tipos, inteiros 1 a 7) Designação do tipo de cobertura florestal.

---

### 1.4.2. Carregar Dados

In [12]:
path   = '' #'/content/drive/MyDrive/kaggle/Tabular Playground Series/2021/12 - Dezembro/'
target = 'Cover_Type'

In [ ]:
df1_train     = pd.read_csv(path + 'Data/train.csv')
df1_test      = pd.read_csv(path + 'Data/test.csv')
df_submission = pd.read_csv(path + 'Data/sample_submission.csv')

df1_train.drop('Id', axis=1, inplace=True)
df1_test.drop('Id', axis=1, inplace=True)

df1_train.shape, df1_test.shape, df_submission.shape

In [ ]:
df1_train.head()

In [ ]:
df1_test.head()

# <div class="alert alert-success"> 2. PROCESSAMENTO </div> 

Neste processo vamos fazer a exclusão de algumas variáveis e fazer redução dos dataset com a redefinição dos tipos de variáveis.

- Excluir variáveis

Na analise observamos que as variáveis **Soil_Type7** e **Soil_Type15** não tem valores para, sendo assim, podemos fazer a exclusão. 

In [ ]:
df1_train.drop(['Soil_Type7','Soil_Type15'], axis=1, inplace=True)
df1_test.drop(['Soil_Type7','Soil_Type15'], axis=1, inplace=True)

- Excluir algumas linhas com classe raras. 

In [ ]:
#df1_train.drop(df1_train[df1_train[target]==5].index, inplace=True)
#df1_train.drop(df1_train[df1_train[target]==4].index, inplace=True)
#df1_train.drop(df1_train[df1_train[target]==6].index, inplace=True)
#df1_train.drop(df1_train[df1_train[target]==7].index, inplace=True)

- Redução dos datasets

In [ ]:
df1_train = reduce_memory_usage(df1_train)
df1_test  = reduce_memory_usage(df1_test)

- Vamos criar duas variáveis que armazená os tipos.

In [ ]:
feature_float = []
feature_cat   = df1_test.filter(regex=r'Wilderness_Area').columns.to_list() + \
                df1_test.filter(regex=r'Soil_Type').columns.to_list()

for col in df1_test.columns:
    if col not in feature_cat: 
        feature_float.append(col)   

feature_float_test = feature_float.copy()

print('Temos {} variávies numéricas e {} categóricas.'.format(len(feature_float),len(feature_cat)))

# <div class="alert alert-success"> 3. FEATURE ENGINEERING </div> 

## 3.1. Feature Descritivas 
Nesta etapa vamos criar novar variárias com medidas estatísticas.

In [ ]:
def feature_statistic(df, feature_float, feature_cat):
    df['fe_mean']        = df[feature_float].mean(axis=1)   
    df['fe_std']         = df[feature_float].std(axis=1)   
    df['fe_median']      = df[feature_float].median(axis=1)   
    df['fe_var']         = df[feature_float].var(axis=1) 
    df['fe_min']         = df[feature_float].min(axis=1)   
    df['fe_max']         = df[feature_float].max(axis=1)   
    df['fe_skew']        = df[feature_float].skew(axis=1)   
    df['fe_quantile_25'] = df[feature_float].quantile(q=.25, axis=1)
    df['fe_quantile_50'] = df[feature_float].quantile(q=.5, axis=1)
    df['fe_quantile_75'] = df[feature_float].quantile(q=.75, axis=1)
    df['fe_dammy_count'] = df[feature_cat].sum(axis=1)   
    return df

In [ ]:
df1_train = feature_statistic(df1_train, feature_float, feature_cat)  
df1_test  = feature_statistic(df1_test, feature_float, feature_cat)
df1_train.shape, df1_test.shape

In [ ]:
df1_train.filter(regex=r'fe').head()

## 3.2. Aspect

A variável **Aspect** indica a direção e observamos na análise descritiva que a variável tem valores negativos e valores maiores que 360, sendo assim vamos ajustar os valores para a faixa entre 0 e 360, como todos os valares estão entre -360 e 720, vamos adicionar 360 aos ângulos menores que 0 e subrair 360 dos ângulos maiores que 360.


In [ ]:
df1_train["Aspect"][df1_train["Aspect"] < 0]   += 360
df1_train["Aspect"][df1_train["Aspect"] > 359] -= 360

df1_test["Aspect"][df1_test["Aspect"] < 0]   += 360
df1_test["Aspect"][df1_test["Aspect"] > 359] -= 360

## 3.3. Feature baseada em distância

Temos as seguintes variáveis baseadas em distância: 
- Horizontal_Distance_To_Hydrology
- Vertical_Distance_To_Hydrology
- Horizontal_Distance_To_Roadways
- Horizontal_Distance_To_Fire_Points

Em primeiro lugar vamos renomear essas vriáveis e vamos criar novas variáveis.





In [ ]:
new_names = {'Horizontal_Distance_To_Hydrology'   : 'x_dist_hydrlgy',
             'Vertical_Distance_To_Hydrology'     : 'y_dist_hydrlgy',
             'Horizontal_Distance_To_Roadways'    : 'x_dist_rdwys',
             'Horizontal_Distance_To_Fire_Points' : 'x_dist_firepts'}

df1_train.rename(new_names, axis=1, inplace=True)
df1_test.rename(new_names, axis=1, inplace=True)

In [ ]:
feature_float.remove('Horizontal_Distance_To_Hydrology')
feature_float.remove('Vertical_Distance_To_Hydrology')
feature_float.remove('Horizontal_Distance_To_Roadways')
feature_float.remove('Horizontal_Distance_To_Fire_Points')

feature_float.append('x_dist_hydrlgy')
feature_float.append('y_dist_hydrlgy')
feature_float.append('x_dist_rdwys')
feature_float.append('x_dist_firepts')

In [ ]:
def feature_distance(df):
    
    df['fe_Hydro_Fire_1'] = abs(df['x_dist_hydrlgy'] + df['x_dist_firepts'])    
    df['fe_Hydro_Fire_2'] = abs(df['x_dist_hydrlgy'] - df['x_dist_firepts'])    
    df['fe_Hydro_Road_1'] = abs(df['x_dist_hydrlgy'] + df['x_dist_rdwys'])    
    df['fe_Hydro_Road_2'] = abs(df['x_dist_hydrlgy'] - df['x_dist_rdwys'])    
    df['fe_Fire_Road_1']  = abs(df['x_dist_firepts'] + df['x_dist_rdwys'])    
    df['fe_Fire_Road_2']  = abs(df['x_dist_firepts'] - df['x_dist_rdwys'])
    
    #  Manhhattan distance
    df['fe_dist_manh_hydrlgy'] = 1+abs(df['x_dist_hydrlgy']) + np.abs(df['y_dist_hydrlgy'])
    df['fe_dist_manh_hydrlgy'] = 1+abs(df['x_dist_hydrlgy']) + np.abs(df['y_dist_hydrlgy'])
    
    # Euclidean distance to Hydrology
    df["fe_dist_ecldn_hydrlgy"] = 1+abs(df["x_dist_hydrlgy"]**2 + df["y_dist_hydrlgy"]**2)**0.5
    df["fe_dist_ecldn_hydrlgy"] = 1+abs(df["x_dist_hydrlgy"]**2 + df["y_dist_hydrlgy"]**2)**0.5

    df['fe_dist_ecldn_hydrlgy'][df["fe_dist_ecldn_hydrlgy"].isnull()]= df['fe_dist_ecldn_hydrlgy'].median()
    
    return df

In [ ]:
df1_train = feature_distance(df1_train) 
df1_test  = feature_distance(df1_test) 

In [ ]:
df1_train.head().filter(regex=r'fe')

## 3.4. Hillshade

Uma rápida pesquisa no Google sobre Hillshade leva ao seguinte resultado:

> O Hillshading calcula a iluminação da superfície como valores de 0 a 255 com base em uma determinada direção da bússola para o sol (azimute) e uma certa altitude acima do horizonte (altitude). Hillshades são freqüentemente usados ​​para produzir mapas que são visualmente atraentes.

Assim, a sombra de colina é uma representação 3D de um terreno que é usada para obter uma visão sobre sua forma medindo a luminosidade de certos trechos desse terreno que resulta quando uma fonte de luz é lançada em um determinado ângulo.

Em ambos os conjuntos de dados (treino e teste), existem certas observações com valor de sombra superior a 255 ou inferior a 0. Eles devem ser o resultado de um erro de registro e devem ser recolocados com um valor apropriado. Talvez, valores menores que 0 se refiram ao tom mais escuro e substituí-los por 0 deve ser adequado. Da mesma forma, podemos supor que os valores de tom de colina acima de 255 se referem aos tons mais brilhantes e um valor de 255 deve ser uma boa substituição.

In [ ]:
def feature_Hillshade(df):        
    df.loc[df["Hillshade_9am"]  < 0, "Hillshade_9am"]     = 0
    df.loc[df["Hillshade_Noon"] < 0, "Hillshade_Noon"]    = 0
    df.loc[df["Hillshade_3pm"]  < 0, "Hillshade_3pm"]     = 0
    df.loc[df["Hillshade_9am"]  > 255, "Hillshade_9am"]   = 255
    df.loc[df["Hillshade_Noon"] > 255, "Hillshade_Noon"]  = 255
    df.loc[df["Hillshade_3pm"]  > 255, "Hillshade_3pm"]   = 255   
    return df

In [ ]:
df1_train = feature_Hillshade(df1_train)
df1_test  = feature_Hillshade(df1_test)

## 3.5. Soil_Type 
Vamos criar novas variáveis com o somatório dos tipo de solos.

In [ ]:
def feature_Soil_type(df):    
    df['fe_soil_type_sum']        = df.filter(regex=r'Soil_Type').sum(axis=1)
    df['fe_Soil_Type12_32']       = df['Soil_Type32'] + df['Soil_Type12']
    df['fe_Soil_Type23_22_32_33'] = df['Soil_Type23'] + df['Soil_Type22'] + \
                                    df['Soil_Type32'] + df['Soil_Type33']    
    return df

In [ ]:
df1_train = feature_Soil_type(df1_train)
df1_test  = feature_Soil_type(df1_test)

In [ ]:
df1_train.filter(regex=r'fe').head()

## 3.6. Hillshade 

In [ ]:
df1_train['fe_hillshade_mean'] = df1_train.filter(regex=r'Hillshade').mean(axis=1)
df1_test['fe_hillshade_mean']  = df1_test.filter(regex=r'Hillshade').mean(axis=1)

## 3.7. Wilderness_Area

In [ ]:
df1_train['fe_wilderness_area_sum'] = df1_train.filter(regex=r'Wilderness_Area').sum(axis=1)
df1_test['fe_wilderness_area_sum']  = df1_test.filter(regex=r'Wilderness_Area').sum(axis=1)

In [ ]:
df1_train['fe_EVDtH'] = df1_train.Elevation-df1_train.y_dist_hydrlgy
df1_test['fe_EVDtH']  = df1_test.Elevation-df1_test.y_dist_hydrlgy

df1_train['fe_EHDtH'] = df1_train.Elevation-df1_train.x_dist_hydrlgy*0.2
df1_test['fe_EHDtH']  = df1_test.Elevation-df1_test.x_dist_hydrlgy*0.2

## 3.9. Elevation

In [ ]:
df1_train['fe_binned_elevation'] = [math.floor(v/50.0) for v in df1_train['Elevation']]
df1_test['fe_binned_elevation']  = [math.floor(v/50.0) for v in df1_test['Elevation']]

Agora vamos fazer uma reducao nos datasets.

In [ ]:
df1_train = reduce_memory_usage(df1_train)
df1_test  = reduce_memory_usage(df1_test)

In [ ]:
jb.dump(df1_train, path + 'Data/pkl/df_train_fe.pkl.z')
jb.dump(df1_test, path + 'Data/pkl/df_test_fe.pkl.z')

Vamos ver se gerou missing.

In [ ]:
missing_zero_values_table(df1_train)

## 3.10 Gerar PCA
Nesta etapa vamos utilizar a PCA para gerar novas variáveis para os modelos.


In [ ]:
#%%time
feature_pca   = feature_float+feature_cat
pca           = PCA(random_state=12359)
df1_train_pca = pca.fit_transform(df1_train[feature_pca])

skplt.decomposition.plot_pca_component_variance(pca, figsize=(8,6));

In [ ]:
#check for optimal number of features

features = range(pca.n_components_)

plt.figure(figsize=(8,4))
plt.bar(features[:15], pca.explained_variance_[:15], color='lightskyblue')
plt.xlabel('PCA feature')
plt.ylabel('Variance')
plt.xticks(features[:15])
plt.show()

In [ ]:
n_components  = 2
pca           = PCA(n_components=n_components, random_state=123)
pca_feats     = [f'fe_pca_{i}' for i in range(n_components)]

df1_train[pca_feats] = pd.DataFrame(pca.fit_transform(df1_train[feature_pca]))
df1_test[pca_feats]  = pd.DataFrame(pca.fit_transform(df1_test[feature_pca]))

## 3.11 Clustering

In [ ]:
df_train_scaler = scaler_MaxAbsScaler_StandardScaler(df1_train[pca_feats])
df_test_scaler  = scaler_MaxAbsScaler_StandardScaler(df1_test[pca_feats])

In [ ]:
#https://realpython.com/k-means-clustering-python/
#https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py


In [ ]:
%%time 
plt.figure(figsize=(12, 7))
visualizer_1 = KElbowVisualizer(KMeans(random_state=12359), k=(2,10))
visualizer_1.fit(df_train_scaler);
visualizer_1.poof();

In [ ]:
model_kmeans = KMeans(n_clusters=4, random_state=12359)
model_kmeans.fit(df_train_scaler);

clusters_train = model_kmeans.predict(df_train_scaler)
clusters_test  = model_kmeans.predict(df_test_scaler)

df1_train['fe_cluster'] = clusters_train
df1_test['fe_cluster']  = clusters_test

df1_train.shape, df1_test.shape

In [ ]:
del df_train_scaler, df_test_scaler

In [ ]:
df1_train = pd.get_dummies(df1_train, columns=['fe_cluster'])
df1_test = pd.get_dummies(df1_test, columns=['fe_cluster'])

df1_train.shape, df1_test.shape

In [ ]:
df1_train.head()

In [ ]:
jb.dump(df1_train, path + "Data/pkl/df2_nb_02_train.pkl.z")
jb.dump(df1_test,  path + "Data/pkl/df2_nb_02_test.pkl.z")

gc.collect()

# <div class="alert alert-success"> 6. Split Train/Test </div>

In [13]:
df2_train     = jb.load(path+ 'Data/pkl/df2_nb_02_train.pkl.z')
df2_test      = jb.load(path+ 'Data/pkl/df2_nb_02_test.pkl.z')
df_submission = pd.read_csv(path + 'Data/sample_submission.csv')
df2_train.drop(df2_train[df2_train[target]==5].index, inplace=True)

df2_train.shape, df2_test.shape

((3999999, 86), (1000000, 85))

In [14]:
X      = df2_train.drop([target], axis=1)
y      = df2_train[target]
X_test = df2_test

X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size    = 0.2,
                                                      shuffle      = True, 
                                                      #stratify     = y,
                                                      random_state = 0)
del df2_train , df2_test

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape 

((3199999, 85), (3199999,), (800000, 85), (800000,))

# <div class="alert alert-success"> 7. Modelagem </div>

In [15]:
def cross_val_model(model_, model_name_, X_, y_, X_test_, target_, scalers_, fold_=5, path_='', 
                    seed_=12359, feature_scaler_=None, print_report_=False):
    
    n_estimators = model_.get_params()['n_estimators']
         
    
    valid_preds  = {}
    taco         = 76 
    acc_best     = 0
    col_prob     = y_.unique()    
    df_proba     = pd.DataFrame()
    test_preds   = []
    test_pred_proba = np.zeros((1, 1))

    for scaler_ in scalers_: 

        time_start   = datetime.now()
        score        = []
        
                
        if scaler_!=None:
            string_scaler = str(scaler_)        
            string_scaler = string_scaler[:string_scaler.index('(')]
            if feature_scaler_!=None:
                X_tst = X_test_.copy()
                X_tst[feature_scaler_] = pd.DataFrame(scaler_.fit_transform(X_tst[feature_scaler_].copy()), columns=X_test_.columns)
            else: 
                X_tst = pd.DataFrame(scaler_.fit_transform(X_test_.copy()), columns=X_test_.columns)
        else:
            string_scaler = None 
            X_tst = X_test_.copy()
            
        y_pred_test = np.zeros(len(X_test_))

        folds = KFold(n_splits=fold_, shuffle=True, random_state=seed_)
        folds = StratifiedKFold(n_splits=fold_, shuffle=True, random_state=seed_)
        #folds = StratifiedShuffleSplit(n_splits=fold_, test_size=0.5, random_state=seed_)
                
        print('='*taco)
        print('Scaler: {} - n_estimators: {}'.format(string_scaler, n_estimators))
        print('='*taco)

        for fold, (trn_idx, val_idx) in enumerate(folds.split(X_, y_, groups=y_)): 

            time_fold_start = datetime.now()

            # ---------------------------------------------------- 
            # Separar dados para treino 
            # ----------------------------------------------------     
            X_trn, X_val = X_.iloc[trn_idx], X_.iloc[val_idx]
            y_trn, y_val = y_.iloc[trn_idx], y_.iloc[val_idx] 
            
            # ---------------------------------------------------- 
            # Processamento 
            # ----------------------------------------------------     
            if scaler_!=None: 
                if feature_scaler_!=None: 
                    X_trn[feature_scaler_] = pd.DataFrame(scaler_.fit_transform(X_trn[feature_scaler_]), columns=X_trn.columns)
                    X_val[feature_scaler_] = pd.DataFrame(scaler_.fit_transform(X_val[feature_scaler_]), columns=X_trn.columns)  
                else:            
                    X_trn = pd.DataFrame(scaler_.fit_transform(X_trn), columns=X_trn.columns)
                    X_val = pd.DataFrame(scaler_.fit_transform(X_val), columns=X_trn.columns)
                        
            # ---------------------------------------------------- 
            # Treinar o modelo 
            # ----------------------------------------------------            
            model_.fit(X_trn, y_trn,
                       eval_set              = [(X_trn, y_trn), (X_val, y_val)],          
                       early_stopping_rounds = int(n_estimators*.1),
                       verbose               = False)
            
            # ---------------------------------------------------- 
            # Predição 
            # ----------------------------------------------------     
            y_pred_val      = model_.predict(X_val, ntree_limit=model_.best_ntree_limit)    
            y_pred_val_prob = model_.predict_proba(X_val, ntree_limit=model_.best_ntree_limit)    
            y_pred_test    += model_.predict(X_tst) / folds.n_splits

            test_preds.append(model_.predict(X_tst))
            
            df_prob_temp    = pd.DataFrame(y_pred_val_prob, columns=col_prob)
            y_pred_pbro_max = df_prob_temp.max(axis=1)

            df_prob_temp['fold']    = fold+1
            df_prob_temp['id']      = val_idx
            df_prob_temp['y_val']   = y_val.values
            df_prob_temp['y_pred']  = y_pred_val            
            df_prob_temp['y_proba'] = np.max(y_pred_val_prob, axis=1)
            df_prob_temp['scaler']  = str(string_scaler)
                        
            # ---------------------------------------------------- 
            # Score 
            # ---------------------------------------------------- 
            acc   = metrics.accuracy_score(y_val, y_pred_val)
            f1    = metrics.f1_score(y_val, y_pred_val, average='weighted')
            prec  = metrics.precision_score(y_val, y_pred_val, average='macro')
            
            score.append(acc)            

            # ---------------------------------------------------- 
            # Print resultado  
            # ---------------------------------------------------- 
            time_fold_end = diff(time_fold_start, datetime.now())
            msg = '[Fold {}] ACC: {:2.5f} - F1-macro: {:2.5f} - Precision: {:2.5f}  - {}'
            print(msg.format(fold+1, acc, f1, prec, time_fold_end))

        acc_mean = np.mean(score) 
        acc_std  = np.std(score)

        if acc_mean > acc_best:     
            acc_best    = acc_mean           
            model_best  = model_    
            scaler_best = scaler_

        time_end = diff(time_start, datetime.now())   

        print('-'*taco)
        print('[Mean Fold] ACC: {:2.5f} std: {:2.5f} - {}'.format(acc_mean, acc_std, time_end))
        print('='*taco)
        print()

        df_submission[target_] = y_pred_test.astype('int')
        name_file_sub = 'Data/submission/' + model_name_ + '_' + str(scaler_).lower()[:4] + '.csv'
        df_submission.to_csv(path_ + name_file_sub.format(acc_mean), index = False)

        if print_report_:
            y_pred = df_prob_temp[df_prob_temp['scaler']==str(string_scaler)]['y_pred']
            y_vl   = df_prob_temp[df_prob_temp['scaler']==str(string_scaler)]['y_val']
            print(metrics.classification_report(y_vl,y_pred))

    print('-'*taco)
    print('Scaler Best: {}'.format(scaler_best))
    print('Score      : {:2.5f}'.format(acc_best))
    print('-'*taco)
    print()

    return model_ , df_prob_temp.sort_values(by=['scaler','id'])#, test_pred_proba

In [ ]:
%%time 
scalers = [#None,
           StandardScaler(), 
           #RobustScaler(), 
           #MinMaxScaler(), 
           #MaxAbsScaler(), 
           #QuantileTransformer(output_distribution='normal', random_state=0)
          ] 

seed     = 12359
y_unique = y.unique()

sample_weights = compute_sample_weight(class_weight='balanced', y=y)
                 
eval_metric = ['mlogloss','merror']
        
params = {"objective"        : 'multi:softmax', # multi:softprob', #,    
          'eval_metric'      : eval_metric, 
          'learning_rate'    : 0.4,  
          #'sample_weight'    : sample_weights, 
          'num_class'        : len(y.unique()),
          'random_state'     : seed, 
          'n_jobs'           : -1}

if torch.cuda.is_available():           
    params.update({'tree_method' : 'gpu_hist',                    
                   'predictor'   : 'gpu_predictor'})

model, df_proba = cross_val_model(model_         = xgb.XGBClassifier(**params),
                                  model_name_    = 'xgb_baseline_score_{:2.5f}',
                                  X_             = X.head(3000),
                                  y_             = y.head(3000),
                                  X_test_        = X_test,
                                  target_        = target,
                                  scalers_       = scalers,
                                  fold_          = 5, 
                                  path_          = path,
                                  seed_          = seed, 
                                  #feature_scaler_= feature_float_test, 
                                  print_report_  = False
                                  )

Scaler: StandardScaler - n_estimators: 100


In [ ]:
feature_imp_     = model.feature_importances_

df               = pd.DataFrame()
df["Feature"]    = X.columns
df["Importance"] = feature_imp_ / feature_imp_.sum()

df.sort_values("Importance", axis=0, ascending=False, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(13, 70))
bars    = ax.barh(df["Feature"], 
                  df["Importance"], 
                  height    = 0.4,
                  color     = "mediumorchid", 
                  edgecolor = "black")

ax.set_title("Feature importances", fontsize=30, pad=15)
ax.set_ylabel("Feature name", fontsize=20, labelpad=15)
#ax.set_xlabel("Feature importance", fontsize=20, labelpad=15)
ax.set_yticks(df["Feature"])
ax.set_yticklabels(df["Feature"], fontsize=13)
ax.tick_params(axis="x", labelsize=15)
ax.grid(axis="x")

# Adicionando rótulos na parte superior
ax2 = ax.secondary_xaxis('top')
#ax2.set_xlabel("Feature importance", fontsize=20, labelpad=13)
ax2.tick_params(axis="x", labelsize=15)
ax.margins(0.05, 0.01)

# Inverter a direção do eixo y 
plt.gca().invert_yaxis()

<div class="alert alert-info" role="alert">
    
**`NOTA:`** <br>

Como podemos obsevar acima, a média de AUC na validação cruzada foi de 0.73142 e nas submissões do kaggle foram os seguintes scores:

- **`None`**: No treino obtever AUC  de 0.73179 e no kable AUC de 0.73818; <br>
- **`RobustScaler`**: No treino obtever AUC  de 0.73139 e no kable AUC de 0.73534; <br>
- **`QuatileTransfomer`**: No treino obtever AUC 0.73048 e no kable AUC de 0.73531 <br> 

    
</div>

# 8. Feature Import 

### 9. SHAP Values
Explicar as previsões do modelo usando SHAP.

In [ ]:
explainer = shap.Explainer(model_best)

#shap_values = explainer(X)

# <div class="alert alert-success"> 5. Conclusão </div>

<div class="alert alert-info" role="alert">    
Neste notebook criamos novas variáveis utilizando a clusterização e variáveis estatísticas, com a finalidade de ajudar os modelos a identificar padrões no dados para melhor as previsões, na validação realizada com XGB obtivemos uma melhora, porém nas submissões não tivemos melhora. <br>
    
Além da criação de novas variávies, utilizamos o SHAP Values para avaliar o impacto das novas variáveis e antigas nas previsões do modelo XGB, podemos destacar que as variáveis: **fe_mean, fe_max, fe_skew** que foram criadas tem impacto negativo nas predições, assim como outras variáveis puxam a predição para baixo, esse talvez seja o favor de não temos melhoria nas nossas submissões, sendo assim, são variáveis contidas a serem removidas do modelo. <br>
    
Um ponto importante que devemos destacar é que ainda não fizemos os ajuste de parametros e estamos trabalhando apenas com um predito (XGB), sendo assim, essas variáveis podem ter valor preditivos relevantes para outros classificadores, os quais vamos teste em outros notebooks.
    
Nos próximos notebooks vamos fazer o ajuste de parametros para os seguintes classificadores: 
- XGB
- LGBM; 
- CatBoost; 
- Rede Neural; 
- KNN;
- SVN; 
    
<br> 
    
A ideia é criar uma stacking, sendo assim, vamos salvar todos os modelos.     
    
<br>
    
Link notebook kaggle: https://www.kaggle.com/rogeriodelfim/tps-nov-2021-02-feature-engenniring-shap
    
</div>

In [ ]:
https://readthedocs.org/projects/scikit-plot/downloads/pdf/stable/